# Pnoi Anotest

> Notebook for validating the Pnoi dataset and its corresponding annotations, reporting any potential errors or inconsistencies. This script aids in ensuring the quality and integrity of the dataset by identifying issues that may require attention.

### File Nomenclature

![File Nomenclature](../../media/pnoi-file_nomenclature_md.png)


In [180]:
import pandas as pd
import functools
import shutil
import json
import glob
import os

mkdir = lambda p: 0 if os.path.exists(p) else (os.makedirs(p), 1)[1]


### STATE VARIABLES

In [181]:
IS_DRY_RUN = False
IS_DUMMY_PATH = False

### PATHS

In [182]:
EXP_VER = "v11"

REPORTS = "0_pnoi-reports"

ANNOTEST = f"anotest_{EXP_VER}"

DATA_PATH = "DATA_DUMMY" if IS_DUMMY_PATH else "DATA_PNOISTOR"

REPORT_FOLDER = f"../{REPORTS}/{ANNOTEST}"; mkdir(REPORT_FOLDER)

1

### Anotest Class

In [183]:
class Anotest:
    VER = "*"
    SEP = "-"
    META_SEP = "_"
    EXT_SEP = "."
    ANOT_LABELS = ["aa", "ee", "uu", "oo", "ii", "xx", "bb1", "bb2", "bb3", "bb4"]

    fkeys = {
        "APP_CODE": "app_code",
        "SID":"sub_id",
        "FCLASS": "file_class",
        "FID": "file_ID",
        "COMNT": "comment",
        "FFMT": "file_format",
        "FNAME": "file_name",
        "FPATH": "file_path",
        "FMATCH": "file_match"
    }

    def __init__(self, path: str, ver="*") -> None:
        self.VER = ver

        all_files = glob.glob(f"{path}/pnoistor_{self.VER}/*/pnoistor_*")

        self.ALL_FILES_DF = self.make_files_df(all_files)


    def file_dict(self, fpath: str) -> dict:
        _, fname = os.path.split(fpath)

        match_key = fname.split(self.EXT_SEP)[0]

        fitems = fname.replace(self.EXT_SEP, self.SEP, 1).split(self.SEP) 
        fitems += [fname, fpath, match_key]
        fdict = dict(zip(self.fkeys.values(), fitems))
        return fdict
    
    def make_files_df(self, all_files: list) -> pd.DataFrame:
        files = [self.file_dict(fp) for fp in all_files]
        return pd.DataFrame(files)


    # Getters
    def get_subject_list(self) -> list: 
        return self.ALL_FILES_DF["sub_id"].unique().tolist()
    
    def get_subject_files(self, sid: str) -> pd.DataFrame:
        return self.ALL_FILES_DF[self.ALL_FILES_DF["sub_id"] == sid]
    
    def get_subject_metadata(self, sid: str) -> pd.DataFrame:
        return self.ALL_METADATA_DF[self.ALL_METADATA_DF["sub_id"] == sid]
    
    def filter_file_df(self, keys: list) -> pd.DataFrame:

        df = self.ALL_FILES_DF.copy()
        filts = [df[self.fkeys["FNAME"]].str.contains(k) for k in keys]
        filt = functools.reduce(lambda p, c: p & c, filts)

        return df.loc[filt]

    # Checkers
    # ---------------------------------------------------------------------------------------------------------------
    report_columns = ['app_code_data', 'sub_id_data', 'file_name_data', 'file_path_data']

    def check_anotless_audio(self, do_export=False) -> pd.DataFrame:

        AUDIO_FKEYS = ["BA_", "wav|WAV"]
        AUDIO_ANOT_FKEYS = ["BA_", "txt"]

        audio_df = self.filter_file_df(AUDIO_FKEYS)
        anote_df = self.filter_file_df(AUDIO_ANOT_FKEYS)

        fm = audio_df.merge(anote_df, how="left", on=[self.fkeys["FMATCH"]], suffixes=("_data", "_anot"))

        k = self.fkeys["FNAME"]

        result = fm.loc[fm[f"{k}_anot"].isna()][self.report_columns]
        
        # export
        if do_export:
            result.to_csv(f"{REPORT_FOLDER}/anotless_audio.csv", index=False)
        
        return result
    
    def check_anotless_pft(self, do_export=False) -> pd.DataFrame:

        PFT_FKEYS = ["PFT_", "pdf"]
        PFT_ANOT_FKEYS = ["PFT_", "tsv|csv"]

        audio_df = self.filter_file_df(PFT_FKEYS)
        anote_df = self.filter_file_df(PFT_ANOT_FKEYS)

        fm = audio_df.merge(anote_df, how="left", on=[self.fkeys["FMATCH"]], suffixes=("_data", "_anot"))

        k = self.fkeys["FNAME"]

        result = fm.loc[fm[f"{k}_anot"].isna()][self.report_columns]
        
        # export
        if do_export:
            result.to_csv(f"{REPORT_FOLDER}/anotless_pft.csv", index=False)
        
        return result
    
    def check_missing_metadata(self, do_export=False) -> pd.DataFrame:
        META_FKEYS = ["META", ".meta.json"]

        metadata_df = self.filter_file_df(META_FKEYS)

        all_subjects = self.get_subject_list()
        _subjects = metadata_df["sub_id"].unique().tolist()

        missing_subjects = list(set(all_subjects) - set(_subjects))
        
        result = missing_subjects
        
        # export
        if do_export:
            with open(f"{REPORT_FOLDER}/missing_metadata.json", "w") as f:
                json.dump(missing_subjects, f)

        return result
    
    def check_pairless_breath(self, do_export=False):

        AUDIO_ANOT_FKEYS = ["BA_", "txt"]

        audio_anot_file_df = self.filter_file_df(AUDIO_ANOT_FKEYS)
        aud_anot = [
            self.read_aud_anot(af)
            for _, af in audio_anot_file_df.iterrows()
        ]
        
        audio_anot_df = pd.concat(aud_anot).copy()

        filt = ~audio_anot_df["label"].isin(self.ANOT_LABELS)

        pairless = [audio_anot_df[filt]]
        for file_name in audio_anot_df["file_name"].unique().tolist():
            try:
                filt = (audio_anot_df["file_name"] == file_name) 
                filt &= (audio_anot_df["label"].str.contains("ii|xx")) 

                df = audio_anot_df.loc[filt].reset_index()
                df["shift_cat_label"] = df["label"] + df["label"].shift(-1)
                df["shift_dif_dur"] = df["end"].shift(-1) - df["start"]

                df_even = df.iloc[::2]
                result = df_even[df_even["shift_cat_label"] != 'iixx']
                pairless.append(result.head(1))
            except Exception as e:
                print("file name", file_name)
                
                print(e)

        result = pd.concat(pairless)
        
        # export
        if do_export:
            result.to_csv(f"{REPORT_FOLDER}/pairless_breath.csv", index=False)
        
        return result
    # ---------------------------------------------------------------------------------------------------------------
    

    
    def read_aud_anot(self, finfo: pd.Series):
        try:
            df = pd.read_csv(finfo[self.fkeys["FPATH"]], sep="\t", names=["start", "end", "label"])
            df["line_number"] = range(1, len(df) + 1)
            df["dur"] = df["end"] - df["start"]
        except Exception as e:
            fp = finfo[self.fkeys["FPATH"]]
            print("file name", f"./{fp}")
                
            print(e)
            
        return pd.concat([finfo.to_frame().T]*len(df), ignore_index=True, axis=0).join(df)
    
    

anotest = Anotest(DATA_PATH, ver="*")
anotest.ALL_FILES_DF


,app_code,sub_id,file_class,file_ID,comment,file_format,file_name,file_path,file_match
0,pnoistor_may2023,shivaganga_46c140d9,LBA_after_LL,4836,comnt,wav,pnoistor_may2023-shivaganga_46c140d9-LBA_after...,DATA_PNOISTOR/pnoistor_may2023/shivaganga_46c1...,pnoistor_may2023-shivaganga_46c140d9-LBA_after...
1,pnoistor_may2023,shivaganga_46c140d9,PFT_after,a0ed,comnt,tsv,pnoistor_may2023-shivaganga_46c140d9-PFT_after...,DATA_PNOISTOR/pnoistor_may2023/shivaganga_46c1...,pnoistor_may2023-shivaganga_46c140d9-PFT_after...
2,pnoistor_may2023,shivaganga_46c140d9,LBA_before_RL,f2ec,comnt,wav,pnoistor_may2023-shivaganga_46c140d9-LBA_befor...,DATA_PNOISTOR/pnoistor_may2023/shivaganga_46c1...,pnoistor_may2023-shivaganga_46c140d9-LBA_befor...
3,pnoistor_may2023,shivaganga_46c140d9,LBA_after_LL,4836,comnt,txt,pnoistor_may2023-shivaganga_46c140d9-LBA_after...,DATA_PNOISTOR/pnoistor_may2023/shivaganga_46c1...,pnoistor_may2023-shivaganga_46c140d9-LBA_after...
4,pnoistor_may2023,shivaganga_46c140d9,VBA_after,caee,phn,txt,pnoistor_may2023-shivaganga_46c140d9-VBA_after...,DATA_PNOISTOR/pnoistor_may2023/shivaganga_46c1...,pnoistor_may2023-shivaganga_46c140d9-VBA_after...
...,...,...,...,...,...,...,...,...,...
664,pnoistor_apr2023,anitajoby_8bf5d711,PFT_after,acf4,comnt,pdf,pnoistor_apr2023-anitajoby_8bf5d711-PFT_after-...,DATA_PNOISTOR/pnoistor_apr2023/anitajoby_8bf5d...,pnoistor_apr2023-anitajoby_8bf5d711-PFT_after-...
665,pnoistor_apr2023,anitajoby_8bf5d711,LBA_before_LU,e544,comnt,wav,pnoistor_apr2023-anitajoby_8bf5d711-LBA_before...,DATA_PNOISTOR/pnoistor_apr2023/anitajoby_8bf5d...,pnoistor_apr2023-anitajoby_8bf5d711-LBA_before...
666,pnoistor_apr2023,anitajoby_8bf5d711,LBA_after_RL,5290,comnt,txt,pnoistor_apr2023-anitajoby_8bf5d711-LBA_after_...,DATA_PNOISTOR/pnoistor_apr2023/anitajoby_8bf5d...,pnoistor_apr2023-anitajoby_8bf5d711-LBA_after_...
667,pnoistor_apr2023,anitajoby_8bf5d711,LBA_after_RU,2d46,comnt,wav,pnoistor_apr2023-anitajoby_8bf5d711-LBA_after_...,DATA_PNOISTOR/pnoistor_apr2023/anitajoby_8bf5d...,pnoistor_apr2023-anitajoby_8bf5d711-LBA_after_...


In [184]:
anotest.check_anotless_audio(do_export=True)

,app_code_data,sub_id_data,file_name_data,file_path_data
16,pnoistor_may2023,mohammadroshan_269b6326,pnoistor_may2023-mohammadroshan_269b6326-VBA_a...,DATA_PNOISTOR/pnoistor_may2023/mohammadroshan_...
17,pnoistor_may2023,mohammadroshan_269b6326,pnoistor_may2023-mohammadroshan_269b6326-VBA_b...,DATA_PNOISTOR/pnoistor_may2023/mohammadroshan_...
18,pnoistor_may2023,mohammadroshan_269b6326,pnoistor_may2023-mohammadroshan_269b6326-LBA_b...,DATA_PNOISTOR/pnoistor_may2023/mohammadroshan_...
85,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_after_LL-bed...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
86,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_before_LU-d4...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
87,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_before_RU-3b...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
88,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_before_RL-5d...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
89,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_after_LU-004...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
90,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_after_RL-2d4...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...
91,pnoistor_dec01,manju_e54090da,pnoistor_dec01-manju_e54090da-LBA_after_RU-867...,DATA_PNOISTOR/pnoistor_dec01/manju_e54090da/pn...


In [185]:
anotest.check_anotless_pft(do_export=True)

,app_code_data,sub_id_data,file_name_data,file_path_data
23,pnoistor_oct07,jesurajabandekar_7273cc8f,pnoistor_oct07-jesurajabandekar_7273cc8f-PFT_b...,DATA_PNOISTOR/pnoistor_oct07/jesurajabandekar_...


In [186]:
anotest.check_missing_metadata(do_export=True)

[]

In [187]:
df = anotest.check_missing_metadata(do_export=True)

In [188]:
for row in anotest.check_pairless_breath(do_export=True).iterrows():
    
    print("'",row[1]["label"], "'", sep="")
    print("line no. ", row[1]["line_number"])
    file_p =  row[1]["file_path"]
    print(f"./{file_p}")
    print(row[1]["shift_cat_label"])
    print()
    

'bb1 '
line no.  1
./DATA_PNOISTOR/pnoistor_jun2023/shreya_2225d70c/pnoistor_jun2023-shreya_2225d70c-LBA_before_LU-3998-comnt.txt
nan

'bb1 '
line no.  1
./DATA_PNOISTOR/pnoistor_jun2023/prathammishra_f569f078/pnoistor_jun2023-prathammishra_f569f078-LBA_before_LU-319c-comnt.txt
nan

'xx '
line no.  12
./DATA_PNOISTOR/pnoistor_jun2023/prathammishra_f569f078/pnoistor_jun2023-prathammishra_f569f078-VBA_before-60a0-comnt.txt
nan

'ii '
line no.  22
./DATA_PNOISTOR/pnoistor_jun2023/prathammishra_f569f078/pnoistor_jun2023-prathammishra_f569f078-VBA_before-60a0-comnt.txt
nan

'aa '
line no.  24
./DATA_PNOISTOR/pnoistor_jun2023/prathammishra_f569f078/pnoistor_jun2023-prathammishra_f569f078-VBA_before-60a0-comnt.txt
nan

'ii'
line no.  9
./DATA_PNOISTOR/pnoistor_jun2023/nikhath_6a3e622e/pnoistor_jun2023-nikhath_6a3e622e-VBA_before-e171-comnt.txt
iiii

'xx'
line no.  44
./DATA_PNOISTOR/pnoistor_jun2023/shreya_2225d70c/pnoistor_jun2023-shreya_2225d70c-VBA_before-80f8-comnt.txt
xxii

'ii'
line no.